<a href="https://colab.research.google.com/github/meghanak-27/skinDisease/blob/main/update1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
# from PIL import Image
import cv2
import numpy as np

In [2]:
class driveFile:
  def __init__(self, mountpath):
    self.path = mountpath
    drive.mount(self.path)
  def dataload(self,folder_path):
    img=[]
    label=[]

    self.path1=os.path.join(self.path,folder_path)

    if os.path.exists(self.path1):
      for i in os.listdir(self.path1):
        self.dis_path=os.path.join(self.path1,i)
        # /content/drive/MyDrive/train_set/BA- cellulitis
        if os.path.exists(self.dis_path) and '.DS_Store' not in self.dis_path:
          for j in os.listdir(self.dis_path):
           if j.endswith(('.jpg', '.png', '.jpeg')):
              self.img_path = os.path.join(self.dis_path, j)
              loaded_img= self.imgload(self.img_path)
              if loaded_img is not None:
                img.append(loaded_img)
                label.append(i)
              else:
                 print("Skipping non-image file:", j)
          else:
              print("Skipping invalid path or .DS_Store:", self.dis_path)
              continue
      return np.array(img), np.array(label)
    else:
      return "patherror","patherror"



  def imgload(self,imgpath):
    img=cv2.imread(imgpath)
    if img is None:
      print(f"Failed to load image at {imgpath}")
      return None
    img=cv2.resize(img,(20,20))
    # v imp, usually 224*224 but here 20*20 "downsampling or using a smaller image resolution before flattening it."
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=img.astype('float32')/255.0
    img_array=np.array(img)
    return img_array


In [4]:
drive=driveFile('/content/drive')

# drive.dataload('')

Mounted at /content/drive


In [5]:
ti,tl=drive.dataload('MyDrive/train_set')
# /content/drive/MyDrive/train_set/BA- cellulitis/BA- cellulitis (1).jpeg

Skipping invalid path or .DS_Store: /content/drive/MyDrive/train_set/BA- cellulitis
Skipping invalid path or .DS_Store: /content/drive/MyDrive/train_set/FU-athlete-foot


In [6]:
ti.shape
# 321 images, 244*244 pixels grayscale



(197, 20, 20)

In [7]:
import pandas as pd


In [8]:

ti[0].shape
# 224*224

(20, 20)

In [9]:
ti_flat = [img.flatten() for img in ti]
# The perceptron (and most neural networks) operates on vectors, not matrices.
df = pd.DataFrame({
    'image': ti_flat,  # Image data (NumPy arrays)
    'label': tl   # Corresponding labels
})

disease_counts = df['label'].value_counts()
print(disease_counts)


label
FU-athlete-foot    101
BA- cellulitis      96
Name: count, dtype: int64


In [10]:
ti_flat[0].shape
ti_flat=np.array(ti_flat)
ti_flat.shape

(197, 400)

In [11]:
w=np.random.randn(400,3)*0.01
# (i\p features,o\p classes)
b = np.zeros((1, 3))


In [12]:
sampled_df = df.groupby('label').apply(lambda x: x.sample(n=20, random_state=42) if len(x) > 20 else x)
sampled_df = sampled_df.reset_index(drop=True)

<ipython-input-12-63a0030ad8ff>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby('label').apply(lambda x: x.sample(n=20, random_state=42) if len(x) > 20 else x)


In [13]:
images20=np.array(sampled_df['image'])
images20.size

40

In [14]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def forward_propagation(X, W, b):
    Z = np.dot(X, W) + b
    A = sigmoid(Z)
    return A  # A is the output of the sigmoid activation (probabilities)
def compute_cost(A, Y):
    m = Y.shape[0]  # Number of training examples
    # Cross-entropy cost
    cost = -np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A)) / m
    return cost
